# Dependencies Load


Installing all the dependencies for creating OpenAI-powered-RAG chatbot

In [ ]:
! pip install python-dotenv langchain langchain-openai langchain-chroma langchain_community openai unstructured tiktoken lark "unstructured[docx]" loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

# Implementation
Import the dependencies


In [ ]:
import os
import openai
import sqlite3
from sqlite3 import Error
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredWordDocumentLoader
from langchain.memory import ConversationTokenBufferMemory
from langchain_core.callbacks import FileCallbackHandler, StdOutCallbackHandler
from langchain_core.output_parsers import StrOutputParser
from loguru import logger
from unstructured.partition.docx import partition_docx

Set the logger

In [ ]:
logfile = "output1.log"

logger.add(logfile, colorize=True, enqueue=True)
handler_1 = FileCallbackHandler(logfile)
handler_2 = StdOutCallbackHandler()

Set environment variables


In [ ]:
os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'
# Ensure OpenAI API key is set
openai.api_key = "OPENAI_API_KEY"

Define the RAG functionality using LLM and Company Docs


In [ ]:
class RAG:
    def __init__(self, docs_dir: str, n_retrievals: int = 6, chat_max_tokens: int = 300, model_name="gpt-3.5-turbo", temperature: float = 0.7):
        self.__model = self.__set_llm_model(model_name, temperature)
        self.__docs_list = self.__get_docs_list(docs_dir)
        self.db = self.__set_chroma_db()
        self.__retriever = self.__set_retriever(k=n_retrievals)
        self.__chat_history = self.__set_chat_history(max_token_limit=chat_max_tokens)

    def __set_llm_model(self, model_name="gpt-3.5-turbo", temperature: float = 0.7):
        return ChatOpenAI(model_name=model_name, temperature=temperature)

    def __get_docs_list(self, docs_dir: str) -> list:
        loader = DirectoryLoader(docs_dir, recursive=True, show_progress=True, glob="*.docx", loader_cls=UnstructuredWordDocumentLoader)
        docs_list = loader.load_and_split()
        return docs_list

    def __set_chroma_db(self):
        embeddings = OpenAIEmbeddings()
        db = Chroma.from_documents(self.__docs_list, embeddings)
        return db

    def __set_retriever(self, k: int = 4):

        # Similarity search
        # query = "What did the president say about Ketanji Brown Jackson"
        # docs = db.similarity_search(query)
        metadata_field_info = [
            {
                "name": "source",
                "description": "The directory path where the document is located",
                "type": "string"
            },
        ]
        document_content_description = "Personal documents"
        retriever = self.db.as_retriever(
            search_kwargs={"k": k}
          )
        return retriever

    def __set_chat_history(self, max_token_limit: int = 300):
        return ConversationTokenBufferMemory(llm=self.__model, max_token_limit=max_token_limit, return_messages=True)

    def ask(self, question: str):
      # Perform similarity search with Chroma vector database
        relevant_docs = self.db.similarity_search(question)

        if relevant_docs:
            # If relevant documents are found, use them to generate a response
            context = "\n".join([doc.page_content for doc in relevant_docs])
            messages = [
                SystemMessage(content="You are a helpful assistant."),
                HumanMessage(content=f"Context: {context}\n\nQuestion: {question}")
            ]
            response = self.__model(messages)
            response = response.content

            logger.info(f"Accessed: RAG \nResponse: {response}")
            #final_response = StrOutputParser().parse(response)
            #logger.info(final_response)

        else:
            # If no relevant documents are found, generate a response directly using the language model
            response = self.__model([HumanMessage(content=question)])
            response = response.content

            logger.info(f"Accessed: Q/A model \nResponse: : {response}")

        return response


Setting Database


In [ ]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        drop_table(conn)
        create_table(conn)
        add_sample_customers(conn)
        return conn
    except Error as e:
        print(e)
    return conn


In [ ]:
# Drop the existing customers table if it exists
def drop_table(conn):
    try:
        sql_drop_customers_table = """ DROP TABLE IF EXISTS customers; """
        c = conn.cursor()
        c.execute(sql_drop_customers_table)
    except Error as e:
        print(e)

In [ ]:
# Create the customers table
def create_table(conn):
    try:
        sql_create_customers_table = """ CREATE TABLE IF NOT EXISTS customers (
                                            username text NOT NULL,
                                            password text NOT NULL,
                                            name text NOT NULL,
                                            email text NOT NULL,
                                            address text NOT NULL,
                                            account text NOT NULL,
                                            serviceUse text NOT NULL
                                        ); """
        c = conn.cursor()
        c.execute(sql_create_customers_table)
    except Error as e:
        print(e)


In [ ]:
# authenticate customers for personalized tasks
def authenticate_customer(conn, account_number, password):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM customers WHERE account = ? AND password = ?", (account_number, password))
        customer = cursor.fetchone()
        if customer:
            return True
        else:
            return False
    except Error as e:
        print("Error:", e)
        return False


In [ ]:
def format_response(response):
    formatted_response = ""
    for key, value in response.items():
        formatted_response += f"{key}: {value}\n"
    return formatted_response

Add sample customers in database for testing

In [ ]:
# Add sample customers to the database
def add_sample_customers(conn):
    try:
        sql_insert_customer = """ INSERT INTO customers (username, password, name, email, address, account, serviceUse)
                                  VALUES (?, ?, ?, ?, ?, ?, ?) """
        customers = [
            ('johndoe123','secretkey76','John Doe', 'john@yahoo.com', '123 Elm St', '234567','Enterprise Plan'),
            ('janes_acc2','janes_pass45','Jane Doe', 'jane@gmail.com', '456 Oak St', '739200','Family Standard')
        ]
        c = conn.cursor()
        c.executemany(sql_insert_customer, customers)
        conn.commit()
    except Error as e:
        print(e)

Define DB Query function

In [ ]:
def query_customer(conn, username):
    print("Verification required")
    while True:
        account_number = input("Enter your account number: ")
        password = input("Enter your password: ")
        # Authenticate the customer
        if authenticate_customer(conn, account_number, password):
            print("Authentication successful!")
            break
        else:
            print("Invalid credentials. Please try again.")
            break
        return None

    try:
        sql_select_customer = f""" SELECT * FROM customers WHERE username = ? """
        c = conn.cursor()
        c.execute(sql_select_customer, (username,))
        return c.fetchone()
    except Error as e:
        print(e)
        return None

Define fetch and update for DB


In [ ]:
def update_customer(conn, username, field, new_value):
    print("Verification required")
    while True:
        account_number = input("Enter your account number: ")
        password = input("Enter your password: ")
        # Authenticate the customer
        if authenticate_customer(conn, account_number, password):
            print("Authentication successful!")
            break
        else:
            print("Invalid credentials. Please try again.")
            break
        return None
    try:
        sql_update_customer = f""" UPDATE customers
                                   SET {field} = ?
                                   WHERE username = ? """
        c = conn.cursor()
        c.execute(sql_update_customer, (new_value, username))
        conn.commit()
    except Error as e:
        print(e)
        return False
    return True

In [ ]:
# Initialize database connection
conn = create_connection("customers.db")



Call the functionalities in the chatbot function


In [ ]:
def chatbot(user_input, conn, rag):
    if user_input.startswith("get customer"):
        username = user_input.split(" ")[-1]
        customer = query_customer(conn, username)
        if customer:
            response = {
                "username": customer[0],
                "password": customer[1],
                "name": customer[2],
                "email": customer[3],
                "address": customer[4],
                "account": customer[5],
                "serviceUse": customer[6]
            }
            logger.info(f"Accessed: DB GET \nResponse: {format_response(response)}" )
            return format_response(response)
        else:
            return {"error": "Customer not found"}
    elif user_input.startswith("update customer"):
        parts = user_input.split(" ")
        username = parts[2]
        field = parts[3]
        print(username, field)
        new_value = " ".join(parts[4:])
        success = update_customer(conn, username, field, new_value)
        if success:
            logger.info("Accessed: DB UPDATE \nResponse: Customer updated")
            return {"status": "Customer updated"}
        else:
            logger.info("Accessed: DB UPDATE \nResponse: Customer not found or update failed")
            return {"error": "Customer not found or update failed"}
    else:
        response = rag.ask(user_input)
        return response

Assign parameters of LLM Model and Documents to be used in RAG instance

In [ ]:
# Initialize RAG instance
rag = RAG(
    docs_dir='/content/Documents',  # Name of the directory where the documents are located
    n_retrievals=1,  # Number of documents returned by the search
    chat_max_tokens=300,  # Maximum number of tokens that can be used in chat memory
    temperature=1.2,  # How creative the response will be
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
  0%|          | 0/6 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 6/6 [00:02<00:00,  2.81it/s]
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the cl

Run the chatbot

In [ ]:
print("\nType 'exit' to exit the program.")
while True:
    user_input = input("Enter your message: ")
    if user_input.lower() == "exit":
        break
    response = chatbot(user_input, conn, rag)
    print('Response:', response)



Type 'exit' to exit the program.
Enter your message: hi


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
2024-05-25 12:01:25.883 | INFO     | __main__:ask:57 - Accessed: RAG 
Response: Hello! How can I assist you today?


Response: Hello! How can I assist you today?
Enter your message: get customer johndoe123
Verification required
Enter your account number: 234567
Enter your password: secretkey76


2024-05-25 12:01:54.907 | INFO     | __main__:chatbot:15 - Accessed: DB GET 
Response: username: johndoe123
password: secretkey76
name: John Doe
email: john@yahoo.com
address: 123 Elm St
account: 234567
serviceUse: Enterprise Plan



Authentication successful!
Response: username: johndoe123
password: secretkey76
name: John Doe
email: john@yahoo.com
address: 123 Elm St
account: 234567
serviceUse: Enterprise Plan

Enter your message: tell me about your services


2024-05-25 12:02:17.402 | INFO     | __main__:ask:57 - Accessed: RAG 
Response: Certainly! Here is an overview of the services and products offered by Mob5GService:

**1. Mobile Services:**

- **Mobile Plans:**
  - Individual Plans: Basic, Standard, Premium.
  - Family Plans: Family Basic, Family Standard, Family Premium.
  - Business Plans: Small Business Plan, Enterprise Plan.

- **Prepaid Plans:**
  - Pay-as-You-Go.
  - Monthly Prepaid.

- **International Plans:**
  - Roaming Plans.
  - International Calling.

**2. Internet Services:**

- **Home Internet:** Basic, Standard, Premium.
- **Mobile Internet:**
  - Hotspot Plans: Basic, Standard, Premium.
  - Portable Wi-Fi Devices.
- **Business Internet:** Small Business Internet, Enterprise Internet.

**3. Devices:**

- **Smartphones:** Latest models from top brands with financing options.
- **Tablets:** Wide range suitable for personal & professional use.
- **Accessories:** Cases, chargers, headphones.
- **Mobile Hotspots and Routers:*

Response: Certainly! Here is an overview of the services and products offered by Mob5GService:

**1. Mobile Services:**

- **Mobile Plans:**
  - Individual Plans: Basic, Standard, Premium.
  - Family Plans: Family Basic, Family Standard, Family Premium.
  - Business Plans: Small Business Plan, Enterprise Plan.

- **Prepaid Plans:**
  - Pay-as-You-Go.
  - Monthly Prepaid.

- **International Plans:**
  - Roaming Plans.
  - International Calling.

**2. Internet Services:**

- **Home Internet:** Basic, Standard, Premium.
- **Mobile Internet:**
  - Hotspot Plans: Basic, Standard, Premium.
  - Portable Wi-Fi Devices.
- **Business Internet:** Small Business Internet, Enterprise Internet.

**3. Devices:**

- **Smartphones:** Latest models from top brands with financing options.
- **Tablets:** Wide range suitable for personal & professional use.
- **Accessories:** Cases, chargers, headphones.
- **Mobile Hotspots and Routers:** Devices for portable Wi-Fi access.

**4. Value-Added Services:**

- 